In [11]:
import os
import requests
import time
import json
from collections import defaultdict
from typing import Dict, List, Tuple, Optional


LEETCODE_SESSION = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzb2NpYWxhY2NvdW50X3N0YXRlIjpbeyJuZXh0IjoiLyIsInByb2Nlc3MiOiJsb2dpbiIsInNjb3BlIjoiIiwiYXV0aF9wYXJhbXMiOiIifSwiTzg4WEZJMEpnMkhzIl0sIl9hdXRoX3VzZXJfaWQiOiI4NDAxMzM5IiwiX2F1dGhfdXNlcl9iYWNrZW5kIjoiZGphbmdvLmNvbnRyaWIuYXV0aC5iYWNrZW5kcy5Nb2RlbEJhY2tlbmQiLCJfYXV0aF91c2VyX2hhc2giOiI4NzYwZDgyNTA5MmQ4ZjVmNDQ4YzFlNDc2NjQ3ZjlhZjAzMzQ4MmFlYzQ3NjI0NWM3NDNlOWM0YWJkNDNmMWNkIiwiaWQiOjg0MDEzMzksImVtYWlsIjoiIiwidXNlcm5hbWUiOiJzZXJlbmUta2lsYnlncHAiLCJ1c2VyX3NsdWciOiJzZXJlbmUta2lsYnlncHAiLCJhdmF0YXIiOiJodHRwczovL2Fzc2V0cy5sZWV0Y29kZS5jbi9hbGl5dW4tbGMtdXBsb2FkL2RlZmF1bHRfYXZhdGFyLnBuZyIsInBob25lX3ZlcmlmaWVkIjp0cnVlLCJpcCI6IjE0LjEyNy40LjM4IiwiX3RpbWVzdGFtcCI6MTc1Nzk0NjcyMC45MjYyMzA0LCJleHBpcmVkX3RpbWVfIjoxNzYwNDY4NDAwLCJ2ZXJzaW9uX2tleV8iOjAsImRldmljZV9pZCI6ImM5Yjc5OTg1OTk5YTMwZDI1MWViOGVmMGUwNWM2M2FkIn0.0deG8AJZiSrTU1ps6FaZQJMlCmWboV0Z_8iHyntpyQo"
LEETCODE_CSRF = "ep9aVkcenmY9WjPXnFjx5ovCniHcw49TV4hA5IKQieGXWZpFTP1FjBRR6tbwwBzB"

class LeetCodeSubmitter:
    def __init__(self, solutions_dir: str, record_prefix: str = "submission_record"):
        """
        Initialize LeetCode submitter for Erlang solutions.

        Args:
            solutions_dir (str): Directory containing Erlang solution files.
            record_prefix (str): Prefix for submission record files.
        """
        self.solutions_dir = solutions_dir
        self.record_prefix = record_prefix
        self.session = requests.Session()

        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            'Content-Type': 'application/json',
            'X-CSRFToken': LEETCODE_CSRF,
            'Origin': 'https://leetcode.cn',
            'Referer': 'https://leetcode.cn/problems/'
        }

        self.session.cookies.set("LEETCODE_SESSION", LEETCODE_SESSION)
        self.session.cookies.set("csrftoken", LEETCODE_CSRF)

        self.total_problems = 0
        self.submitted_problems = 0
        self.accepted_problems = 0
        self.failed_problems = 0
        self.skipped_problems = 0
        self.problem_details = []

        self.record_index = 0
        self.record_file = f"{self.record_prefix}_{self.record_index}.erlang.txt"
        self.submission_records = self._load_submission_records()

        print("LeetCode session initialized. Ready to submit Erlang solutions...")

    def _rotate_record_file(self):
        """Rotate record file to avoid oversized logs."""
        if os.path.exists(self.record_file) and os.path.getsize(self.record_file) > 5 * 1024 * 1024:  # 5 MB per record file
            self.record_index += 1
            self.record_file = f"{self.record_prefix}_{self.record_index}.erlang.txt"
            print(f"Rotating record file -> {self.record_file}")

    def _load_submission_records(self) -> Dict[str, str]:
        records = {}
        for idx in range(1000):  # search possible record files
            fname = f"{self.record_prefix}_{idx}.erlang.txt"
            if not os.path.exists(fname):
                break
            with open(fname, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if line and ':' in line:
                        parts = line.split(':', 1)
                        problem_name = parts[0].strip()
                        status = parts[1].strip()
                        records[problem_name] = status
        print(f"Loaded {len(records)} submission records from existing files.")
        return records

    def _get_problem_slug(self, filename: str) -> str:
        basename = os.path.basename(filename)
        name_without_ext = os.path.splitext(basename)[0]
        if '.' in name_without_ext:
            return name_without_ext.split('.', 1)[1]
        elif '-' in name_without_ext:
            parts = name_without_ext.split('-', 1)
            if len(parts) > 1:
                return parts[1]
        return name_without_ext

    def _get_problem_id(self, slug: str) -> Optional[str]:
        max_retries = 5
        retry_count = 0
        while retry_count < max_retries:
            try:
                graphql_url = "https://leetcode.cn/graphql/"
                graphql_payload = {
                    "operationName": "questionData",
                    "variables": {"titleSlug": slug},
                    "query": """
                        query questionData($titleSlug: String!) {
                            question(titleSlug: $titleSlug) {
                                questionId
                                questionFrontendId
                                title
                                titleSlug
                            }
                        }
                    """
                }
                response = self.session.post(
                    graphql_url,
                    headers=self.headers,
                    json=graphql_payload,
                    timeout=10
                )
                if response.status_code == 200:
                    data = response.json()
                    q = data.get("data", {}).get("question", {})
                    if q:
                        return q.get("questionId")
                else:
                    print(f"Failed to get problem ID: HTTP {response.status_code}")
            except Exception as e:
                print(f"Error while getting problem ID: {str(e)}")
            retry_count += 1
            time.sleep(2 ** retry_count)
        print(f"Cannot get problem ID for slug '{slug}', skipping.")
        return None

    def _submit_solution(self, problem_slug: str, solution_code: str) -> Tuple[str, str]:
        max_retries = 20
        retry_count = 0
        while retry_count < max_retries:
            try:
                problem_id = self._get_problem_id(problem_slug)
                if not problem_id:
                    return "Failed", "Problem ID not found"
                submit_url = f"https://leetcode.cn/problems/{problem_slug}/submit/"
                payload = {
                    "lang": "erlang",  # Changed from python3 to erlang
                    "question_id": problem_id,
                    "typed_code": solution_code
                }
                response = self.session.post(submit_url, headers=self.headers, json=payload, timeout=30)
                if response.status_code == 200:
                    result = response.json()
                    if "submission_id" in result:
                        sid = result["submission_id"]
                        print(f"Submitted successfully, submission ID: {sid}")
                        return self._check_submission_result(sid)
                else:
                    print(f"Submit failed: HTTP {response.status_code}")
            except Exception as e:
                print(f"Error during submission: {str(e)}")
            retry_count += 1
            time.sleep(min(5 * retry_count, 60))
        return "Failed", "Max retries reached"

    def _check_submission_result(self, submission_id: str) -> Tuple[str, str]:
        max_retries = 30
        retry_count = 0
        while retry_count < max_retries:
            try:
                check_url = f"https://leetcode.cn/submissions/detail/{submission_id}/check/"
                response = self.session.get(check_url, headers=self.headers, timeout=10)
                if response.status_code != 200:
                    return "Pending", f"Check failed: HTTP {response.status_code}"
                result = response.json()
                state = result.get("state", "PENDING")
                if state == "SUCCESS":
                    status = result.get("status_msg", "UNKNOWN")
                    runtime = result.get("status_runtime", "N/A")
                    return status, runtime
                time.sleep(1)
            except Exception as e:
                print(f"Error while checking submission result: {str(e)}")
                time.sleep(2)
            retry_count += 1
        return "Timeout", "Max checks reached"

    def submit_all_solutions(self) -> None:
        start_time = time.time()
        solution_files = [f for f in os.listdir(self.solutions_dir) if f.endswith('.erl')]  # Changed from .py to .erl
        solution_files.sort()
        self.total_problems = len(solution_files)
        if not solution_files:
            print("No Erlang solution files found.")
            return
        print(f"Found {self.total_problems} Erlang solutions.")
        for i, filename in enumerate(solution_files):
            problem_name = os.path.splitext(filename)[0]
            problem_slug = self._get_problem_slug(filename)
            solution_path = os.path.join(self.solutions_dir, filename)
            print(f"\n{'='*50}\nProcessing ({i+1}/{self.total_problems}): {problem_slug} (file: {filename})")
            if problem_name in self.submission_records:
                record_status = self.submission_records[problem_name]
                print(f"Already submitted (status: {record_status}), skipping.")
                self.skipped_problems += 1
                continue
            try:
                with open(solution_path, 'r', encoding='utf-8') as f:
                    solution_code = f.read()
                status, runtime = self._submit_solution(problem_slug, solution_code)
                self._save_submission_record(problem_name, status)
                self.submission_records[problem_name] = status
                self.submitted_problems += 1
                if status == "Accepted":
                    self.accepted_problems += 1
                print(f"Result: {status}, Runtime: {runtime}")
                time.sleep(5)
            except Exception as e:
                print(f"Error while processing {filename}: {str(e)}")
        self.print_statistics(start_time)

    def _save_submission_record(self, problem_name: str, status: str) -> None:
        try:
            self._rotate_record_file()
            with open(self.record_file, 'a', encoding='utf-8') as f:
                f.write(f"{problem_name}: {status}\n")
        except Exception as e:
            print(f"Error while saving submission record: {str(e)}")

    def print_statistics(self, start_time: float) -> None:
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        hours, minutes = divmod(minutes, 60)
        print("\n" + "="*50)
        print("LeetCode Erlang Solutions Submission Report")
        print("="*50)
        print(f"Total problems: {self.total_problems}")
        print(f"Submitted: {self.submitted_problems}")
        print(f"Skipped: {self.skipped_problems}")
        print(f"Accepted: {self.accepted_problems}")
        print(f"Failed: {self.submitted_problems - self.accepted_problems}")
        report_file = "submission_report.json"
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump({
                "total_problems": self.total_problems,
                "submitted_problems": self.submitted_problems,
                "skipped_problems": self.skipped_problems,
                "accepted_problems": self.accepted_problems,
                "problem_details": self.problem_details,
                "elapsed_time": elapsed_time
            }, f, indent=2)
        print(f"Detailed report saved to {report_file}")

def main():
    solutions_dir = "erlang_solutions"  # Changed directory name
    submitter = LeetCodeSubmitter(solutions_dir)
    try:
        submitter.submit_all_solutions()
    except KeyboardInterrupt:
        print("Submission interrupted. Printing current statistics...")
        submitter.print_statistics(time.time())

if __name__ == "__main__":
    main()

Loaded 2518 submission records from existing files.
LeetCode session initialized. Ready to submit Erlang solutions...
Found 2970 Erlang solutions.

Processing (1/2970): two-sum (file: 1-two-sum.erl)
Already submitted (status: Compile Error), skipping.

Processing (2/2970): regular-expression-matching (file: 10-regular-expression-matching.erl)
Already submitted (status: Compile Error), skipping.

Processing (3/2970): minimum-cost-to-merge-stones (file: 1000-minimum-cost-to-merge-stones.erl)
Already submitted (status: Compile Error), skipping.

Processing (4/2970): grid-illumination (file: 1001-grid-illumination.erl)
Already submitted (status: Compile Error), skipping.

Processing (5/2970): find-common-characters (file: 1002-find-common-characters.erl)
Already submitted (status: Compile Error), skipping.

Processing (6/2970): check-if-word-is-valid-after-substitutions (file: 1003-check-if-word-is-valid-after-substitutions.erl)
Already submitted (status: Compile Error), skipping.

Process